In [5]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ------- -------------------------------- 1.8/9.7 MB 12.6 MB/s eta 0:00:01
   -------------- ------------------------- 3.4/9.7 MB 9.2 MB/s eta 0:00:01
   ------------------ --------------------- 4.5/9.7 MB 6.9 MB/s eta 0:00:01
 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Taylor\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import json
import time

# Configuração do driver do Selenium
driver = webdriver.Chrome()  # Certifique-se de ter o ChromeDriver instalado
driver.get("https://my.timetrade.com/AccountManager/user/getUserList?offset=0&max=15&sort=statusid&order=asc")  # Substitua pela URL da página

# Função para extrair o conteúdo do script que contém userData
def extract_user_data(script_text):
    # Regex para localizar o trecho que contém o JSON do userData
    match = re.search(r"var userData = (\{.*?\});", script_text, re.DOTALL)
    if match:
        user_data_json = match.group(1)
        return json.loads(user_data_json)
    return None

# Lista para armazenar todos os usuários
all_users = []

# Iterar por todas as páginas
while True:
    # Aguarde a página carregar (ajuste conforme necessário)
    time.sleep(2)
    
    # Encontre todos os scripts na página
    scripts = driver.find_elements(By.TAG_NAME, "script")
    user_data_found = False
    
    for script in scripts:
        script_text = script.get_attribute("innerHTML")
        if "var userData" in script_text:
            user_data = extract_user_data(script_text)
            if user_data:
                all_users.extend(user_data['items'])
                user_data_found = True
                break
    
    if not user_data_found:
        print("userData não encontrado nesta página.")
        break

    # Tente clicar no botão "Próximo" para navegar para a próxima página
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, ".nextLink")
        next_button.click()
    except Exception as e:
        print("Não há mais páginas para navegar.")
        break

# Fechar o navegador
driver.quit()

# Exibir ou salvar os dados extraídos
for user in all_users:
    print(f"{user['firstname']} {user['lastname']} - {user['pk.emailaddress']}")


Defaulting to user installation because normal site-packages is not writeable
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Taylor\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
